In [3]:
using Symbolics

function HM(sz::Int...; symbol::Symbol)
    (@variables $symbol[map(x -> 1:x, sz)...])[1]
end

function HM(sz::AbstractRange{Int}...; symbol::Symbol)
    (@variables $symbol[sz...])[1]
end

eltypes(::Type{Symbolics.Arr{T,D}}) where {T,D} = (T, D)
eltypes(x::Symbolics.Arr) = eltypes(typeof(x))

Base.similar(A::AbstractArray, T::Type, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(A, T, tuple(map(x -> length(x), dims)...))
    
Base.similar(f::Union{Function,DataType}, dims::Tuple{AbstractRange{I}, Vararg{AbstractRange{I}}}) where {I<:Int} = similar(f, tuple(map(x -> length(x), dims)...))

function printHM(Hx::Symbolics.Arr{T,2}) where {T}
    println("2 method called! ", Hx)
    if any(firstindex.([Hx],[1:2...]') .!= 1)
        r,c = length.(axes(Hx))
        [Hx[i+r*j] for i in 1:r, j in 0:(c-1)]
    else
        [x for x in Hx]
    end
end

function printHM(Hx::Symbolics.Arr{T,D}, dim1=0, dim2=1, dpths=nothing) where {T,D}
    println("general method called! ", Hx)
    if any(firstindex.([Hx],[1:D...]') .!= 1)
        [Hx[i] for i in CartesianIndex(repeat([1],D)...):CartesianIndex(length.(axes(Hx))...)]
    else
        [x for x in Hx]
    end
end

printHM (generic function with 5 methods)

In [4]:
sz = 3; A = HM(sz, sz, symbol=:a)
# A |> printHM
printHM(A)

2 method called! a[1:3,1:3]


3×3 Matrix{Num}:
 a[1, 1]  a[1, 2]  a[1, 3]
 a[2, 1]  a[2, 2]  a[2, 3]
 a[3, 1]  a[3, 2]  a[3, 3]

In [13]:
transMonoid = expand(
    prod(1:sz) do i
        sum(1:sz) do j
            A[i,j]
        end
    end
)
print(transMonoid)

a[1, 1]*a[2, 1]*a[3, 1] + a[1, 3]*a[2, 1]*a[3, 1] + a[1, 1]*a[2, 1]*a[3, 2] + a[1, 3]*a[2, 1]*a[3, 2] + a[1, 1]*a[2, 3]*a[3, 1] + a[1, 1]*a[2, 2]*a[3, 2] + a[1, 1]*a[2, 1]*a[3, 3] + a[1, 1]*a[2, 2]*a[3, 3] + a[1, 1]*a[2, 3]*a[3, 3] + a[1, 2]*a[2, 1]*a[3, 1] + a[1, 2]*a[2, 2]*a[3, 1] + a[1, 2]*a[2, 1]*a[3, 2] + a[1, 2]*a[2, 2]*a[3, 2] + a[1, 2]*a[2, 1]*a[3, 3] + a[1, 2]*a[2, 2]*a[3, 3] + a[1, 2]*a[2, 3]*a[3, 3] + a[1, 3]*a[2, 3]*a[3, 1] + a[1, 3]*a[2, 3]*a[3, 2] + a[1, 3]*a[2, 1]*a[3, 3] + a[1, 1]*a[2, 2]*a[3, 1] + a[1, 3]*a[2, 2]*a[3, 1] + a[1, 3]*a[2, 2]*a[3, 2] + a[1, 3]*a[2, 2]*a[3, 3] + a[1, 2]*a[2, 3]*a[3, 1] + a[1, 1]*a[2, 3]*a[3, 2] + a[1, 2]*a[2, 3]*a[3, 2] + a[1, 3]*a[2, 3]*a[3, 3]

In [14]:
dump(transMonoid)


Num
  val: SymbolicUtils.Add{Real, Int64, Dict{Any, Number}, Nothing}
    coeff: Int64 0
    dict: Dict{Any, Number}
      slots: Array{UInt8}((64,)) UInt8[0x00, 0x00, 0x00, 0x00, 0x01, 0x01, 0x01, 0x01, 0x01, 0x00  …  0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00]
      keys: Array{Any}((64,))
        1: #undef
        2: #undef
        3: #undef
        4: #undef
        5: SymbolicUtils.Mul{Real, Int64, Dict{Any, Number}, Nothing}
          coeff: Int64 1
          dict: Dict{Any, Number}
            slots: Array{UInt8}((16,)) UInt8[0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x01, 0x00]
            keys: Array{Any}((16,))
              1: #undef
              2: #undef
              3: SymbolicUtils.Term{Real, Nothing}
                f: getindex (function of type typeof(getindex))
                arguments: Array{Any}((3,))
                metadata: Nothing nothing
                hash: Base.RefValue{UInt64}
              4: #un

In [37]:
# keys(Symbolics.value(transMonoid).dict)
arguments(Symbolics.value(transMonoid))

27-element Vector{SymbolicUtils.Mul{Real, Int64, Dict{Any, Number}, Nothing}}:
 a[1, 1]*a[2, 1]*a[3, 1]
 a[1, 3]*a[2, 1]*a[3, 1]
 a[1, 1]*a[2, 1]*a[3, 2]
 a[1, 3]*a[2, 1]*a[3, 2]
 a[1, 1]*a[2, 3]*a[3, 1]
 a[1, 1]*a[2, 2]*a[3, 2]
 a[1, 1]*a[2, 1]*a[3, 3]
 a[1, 1]*a[2, 2]*a[3, 3]
 a[1, 1]*a[2, 3]*a[3, 3]
 a[1, 2]*a[2, 1]*a[3, 1]
 a[1, 2]*a[2, 2]*a[3, 1]
 a[1, 2]*a[2, 1]*a[3, 2]
 a[1, 2]*a[2, 2]*a[3, 2]
 ⋮
 a[1, 2]*a[2, 3]*a[3, 3]
 a[1, 3]*a[2, 3]*a[3, 1]
 a[1, 3]*a[2, 3]*a[3, 2]
 a[1, 3]*a[2, 1]*a[3, 3]
 a[1, 1]*a[2, 2]*a[3, 1]
 a[1, 3]*a[2, 2]*a[3, 1]
 a[1, 3]*a[2, 2]*a[3, 2]
 a[1, 3]*a[2, 2]*a[3, 3]
 a[1, 2]*a[2, 3]*a[3, 1]
 a[1, 1]*a[2, 3]*a[3, 2]
 a[1, 2]*a[2, 3]*a[3, 2]
 a[1, 3]*a[2, 3]*a[3, 3]

In [40]:
for graph in arguments(Symbolics.value(transMonoid))
    println(graph)
end

a[1, 1]*a[2, 1]*a[3, 1]
a[1, 3]*a[2, 1]*a[3, 1]
a[1, 1]*a[2, 1]*a[3, 2]
a[1, 3]*a[2, 1]*a[3, 2]
a[1, 1]*a[2, 3]*a[3, 1]
a[1, 1]*a[2, 2]*a[3, 2]
a[1, 1]*a[2, 1]*a[3, 3]
a[1, 1]*a[2, 2]*a[3, 3]
a[1, 1]*a[2, 3]*a[3, 3]
a[1, 2]*a[2, 1]*a[3, 1]
a[1, 2]*a[2, 2]*a[3, 1]
a[1, 2]*a[2, 1]*a[3, 2]
a[1, 2]*a[2, 2]*a[3, 2]
a[1, 2]*a[2, 1]*a[3, 3]
a[1, 2]*a[2, 2]*a[3, 3]
a[1, 2]*a[2, 3]*a[3, 3]
a[1, 3]*a[2, 3]*a[3, 1]
a[1, 3]*a[2, 3]*a[3, 2]
a[1, 3]*a[2, 1]*a[3, 3]
a[1, 1]*a[2, 2]*a[3, 1]
a[1, 3]*a[2, 2]*a[3, 1]
a[1, 3]*a[2, 2]*a[3, 2]
a[1, 3]*a[2, 2]*a[3, 3]
a[1, 2]*a[2, 3]*a[3, 1]
a[1, 1]*a[2, 3]*a[3, 2]
a[1, 2]*a[2, 3]*a[3, 2]
a[1, 3]*a[2, 3]*a[3, 3]


Notes:

- `Base.Filesystem.tempname` exists, so maybe we can create graphs in a similar manner to the equivalent Sage code.
- A recreation of Sage style variables (each entry of a HM is its separate symbol, programatically constructed) may be required at some point, which would require a deeper dive into metaprogramming and macros.